In [3]:
# advanced GAN
# Convolutional layer hacking with Python and Numpy
# https://towardsdatascience.com/convolutional-layer-hacking-with-python-and-numpy-e5f64812ca0c
# # https://medium.com/@ideami
#import the libraries

import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import wandb

def show(tensor, num=25, wamb=0, name=''):
    data = tensor.detach().cpu()
    grid = make_grid(data[:num], nrows=5).permute(1,2,0)

    # optional - Weights and Biases
    if (wandb == 1 and wandbact ==1):
        wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})
    
    plt.imshow(grid.clip(0,1))
    plt.show

n_epochs = 10000
batch_size = 128
lr = 1e-4
z_dim = 200
device='cuda'

cur_step = 0
# critique cycles of training
crit_cycles=5 
gen_losses=[]
crit_losses=[]
show_step=35
save_step=35
# Track stats through weights and biases
wandbact = 1 

wandb.login(key='708ae0ba329cae8bf5ba95fbdbacd445fe02de5f')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Renan/.netrc


True

In [4]:
%%capture
experiment_name = wandb.util.generate_id()

myrun=wandb.init(
    project="wgan",
    group=experiment_name,
    config={
        "optimizer":"adam",
        "model":"wgan gp",
        "epoch":"1000",
        "batch_size":128
    }
)

config=wandb.config

## Sometimes this call to the wandb service may fail or give problems.
## If needed, you can use the Runtime->Restart Runtime option in the menus
## to restart and try again. If you keep having issues, you may set wandbact=0
## to skip temporarily the remote stats. You would also
## need to set wandbactive to 0 within the training loop in
## show(fake, wandbactive=0, name='fake') and show(real, wandbactive=0, name='real').
## However, in general this should work with no problems (try a few times if you have issues)

In [5]:
print(experiment_name)

49462ozz


In [ ]:
# generator model

from turtle import forward


class Generator(nn.Model):
    def __init__(self, z_dim=64, d_dim=16) -> None:
        super(Generator, self).__init__()
        self.z_dim=z_dim

        self.gen = nn.Sequential(
            # ConvTranspose2d: in_channels, out_channels, kernel_size, stride=1, padding=0
            #  Calculating new idth and height: (n-1)*stride -2*padding + ks
            #  n = width or height
            #  ks = kernel size
            #  we begin with a 1x1 image with z_dim number of channels (200)
            nn.ConvTranspose2d(z_dim, d_dim*32, 4, 1, 0), # 4x4 (ch: 200, 512)
            nn.BatchNorm2d(d_dim*32),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*32, d_dim*16, 4, 2, 1), # 8x8 (ch: 512, 256)
            nn.BatchNorm2d(d_dim*16),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*16, d_dim*8, 4, 2, 1), # 16x16 (ch: 256, 128)
            # (n-1)*stride -2*padding + ks = (8-1)*2-2*1+4 = 16
            nn.BatchNorm2d(d_dim*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*8, d_dim*4, 4, 2, 1), # 32x32 (ch: 128, 64)
            nn.BatchNorm2d(d_dim*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*4, d_dim*2, 4, 2, 1), # 64x64 (ch: 64, 32)
            nn.BatchNorm2d(d_dim*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(d_dim*2, 3, 4, 2, 1), # 128x128 (ch: 32, 3)
            nn.Tanh() #produce result in the range from -1 to 1
        )

    def forward(self, noise):
        x=noise.view(len(noise), self.z_dim, 1 , 1) # 128 x 200 x1 x 1
        return self.gen(x)

def gen_noise(num, z_dim, device='cuda'):
    return torch.randn(num, z_dim, device=device) # 128 x 200



In [ ]:
#  Critique model
# Conv2D: in_channels, out_channels, kernel_size, stride=1, apdding=0
# New width and height: # (n+2*pad-ks)//stride +1

class Critic(nn.Module):
    def __init__(self, d_dim=16) -> None:
        super(Critic, self).__init__()
        self.crit = nn.Sequential(
            nn.Conv2d(3,d_dim,4,2,1), # (n+2*pad-ks)//stride +1 = (128+2*1-4)//2+1=64x64 (ch: 3,16)
            nn.InstanceNorm2d(d_dim), # normalizing by instance works best for Critic
            nn.LeakyReLU(0.2), # has a little of a negative slope, help to preserve signal

            nn.Conv2d(d_dim, d_dim*2,4,2,1), # 32x32 (ch: 16,32)
            nn.InstanceNorm2d(d_dim*2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(d_dim*2, d_dim*4,4,2,1), # 16x16 (ch: 32,64)
            nn.InstanceNorm2d(d_dim*4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(d_dim*4, d_dim*8,4,2,1), # 8x8 (ch: 64,128)
            nn.InstanceNorm2d(d_dim*8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(d_dim*8, d_dim*16,4,2,1), # 4x4 (ch: 128,256)
            nn.InstanceNorm2d(d_dim*16),
            nn.LeakyReLU(0.2),

            nn.Conv2d(d_dim*16, 1,4,1,0), # (n+2*pad-ks)//stride +1 = (4+2*0-4)//1+1 = 1x1 (ch: 256,1)

        ) 

    def forward(self, image):
        # iamge: 128 x 3 x128 x 128
        crit_pred = self.critic(image) # 128 x 1 x 1 x 1
        return crit_pred.view(len(crit_pred), -1) # 128 x 1

In [ ]:
#  optional, init your weights in different ways

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight,0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
    
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias,0)
    
# gen = gen.apply(init_weights)
# crit = crit.apply(init_weights)

In [ ]:
# load dataset
# TODO make it work locally
# gdown - Download a large file from Google Drive.
import gdown, zipfile
url = 'https://drive.google.com/'
path='data/celeba'
download_path=f'{path}/img_align_celeba.zip'

if not os.path.exist(path):
    os.makedirs(path)

gdown.download(url, download_path, quiet=False)

with zipfile.ZipFile(download_path, 'r') as ziphandler:
    ziphandler.extractall(path)

#### Alternative download address:
# Celebra gdrive: https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg?resourcekey=0-rJlzl934LzC-Xp28GeIBzQ
# Kaggle: https://www.kaggle.com/jessicali9530/celeba-dataset

In [ ]:
# Dataset, Dataloader, declare gen, crit, test dataset

class Dataset(Dataset):
  def __init__(self, path, size=128, lim=10000):
    self.sizes=[size, size]
    items, labels=[],[]

    for data in os.listdir(path)[:lim]:
      #path: './data/celeba/img_align_celeba'
      #data: '114568.jpg
      item = os.path.join(path,data)
      items.append(item)
      labels.append(data)
    self.items=items
    self.labels=labels


  def __len__(self):
    return len(self.items)

  def __getitem__(self,idx):
    data = PIL.Image.open(self.items[idx]).convert('RGB') # (178,218)
    data = np.asarray(torchvision.transforms.Resize(self.sizes)(data)) # 128 x 128 x 3
    data = np.transpose(data, (2,0,1)).astype(np.float32, copy=False) # 3 x 128 x 128 # from 0 to 255
    data = torch.from_numpy(data).div(255) # from 0 to 1
    return data, self.labels[idx]

## Dataset
data_path='./data/celeba/img_align_celeba'
ds = Dataset(data_path, size=128, lim=10000)

## DataLoader
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True)

## Models
gen = Generator(z_dim).to(device)
crit = Critic().to(device)

## Optimizers
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5,0.9))
crit_opt= torch.optim.Adam(crit.parameters(), lr=lr, betas=(0.5,0.9))

## Initializations
##gen=gen.apply(init_weights)
##crit=crit.apply(init_weights)

#wandb optional
if (wandbact==1):
  wandb.watch(gen, log_freq=100)
  wandb.watch(crit, log_freq=100)

x,y=next(iter(dataloader))
show(x)

In [ ]:
#  gradient penalty calculation

def get_gp(real, fake, crit, alpha, gamma=10):
    mix_images = real * alpha + fake * (1-alpha) # 128 x 3 x 128 x 128
    mix_scores = crit(mix_images) # 128 x 1

    gradient = torch.autograd.grad(
        inputs = mix_images,
        outputs = mix_scores,
        grad_outputs=torch.ones_like(mix_scores),
        retain_graph=True,
        create_graph=True,
  )[0] # 128 x 3 x 128 x 128

    gradient = gradient.view(len(gradient), -1)   # 128 x 49152
    gradient_norm = gradient.norm(2, dim=1) 
    gp = gamma * ((gradient_norm-1)**2).mean()

    return gp


In [ ]:
# Save and Load Checkpoints

root_path='./data/'

def save_checkpoint(name):
    torch.save({
        'epoch': epoch,
        'model_state_dict': gen.state_dict(),
        'optimizer_state_dict': gen_opt.state_dict() 
        }, f"{root_path}G-{name}.pkl"
    )
    torch.save({
        'epoch': epoch,
        'model_state_dict': crit.state_dict(),
        'optimizer_state_dict': crit_opt.state_dict()
        }, f"{root_path}C-{name}.pk1"
    )
    print("saved checkpoint")

def load_checkpoint(name):
    checkpoint = torch.load(f"{root_path}G-{name}.pk1")
    gen.load_state_dict(checkpoint['model_state_dict'])
    gen_opt.load_state_dict(checkpoint['optimizer_state_dict'])

    checkpoint = torch.load(f"{root_path}C-{name}.pkl")
    crit.load_state_dict(checkpoint['model_state_dict'])
    crit_opt.load_state_dict(checkpoint['optimizer_state_dict'])

    print("Loaded checkpoint")

In [ ]:
# training
for epoch in range(n_epochs):
    for real, _ in tqdm(dataloader):
        cur_bs = len(real) # 128
        real = real.to(device)
    
    # CRITIC
    mean_critic_loss = 0
    for _ in range(crit_cycles):
        crit_opt.zero_grad()

        noise=gen_noise(cur_bs, z_dim)
        fake = gen(noise)
        crit_fake_pred = crit(fake.detach())
        crit_real_pred = crit(real)

        alpha = torch.rand(len(real),1,1,1,device=device,requires_grad=True) # 128x1x1x1
        gp = get_gp(real, fake.detach(), crit, alpha)

        crit_loss = crit_fake_pred.mean() - crit_real_pred.mean() + get_gp

        mean_critic_loss += crit_loss.item() / crit_cycles

        crit_loss.backward(retain_graph=True)
        crit_opt.step()

    crit_losses+=[mean_critic_loss]

    #  Generator
    gen_opt.zero_grad()
    noise = gen_noise(cur_bs, z_dim)
    fake = gen(noise)
    crit_fake_pred = crit(fake)

    gen_loss = -crit_fake_pred.mean()
    gen_loss.backward()
    gen_opt.step()

    gen_losses=+[gen_loss.item()]

    # stats
    if (wandbact == 1):
        wandb.log({'Epoch':epoch, 'Step':cur_step, 'Critic loss':mean_critic_loss, 
        'Gen loss':gen_loss})
    
    if cur_step % save_step == 0 and cur_step > 0:
        print("Saving checkpoint: ", cur_step, save_step)
        save_checkpoint("latest")
        
    if (cur_step % show_step == 0 and cur_step >0):
        show(fake, wandbactive=1, name='dake')
        show(real, wandbactive=1, name='real')    

        gen_mean=sum(gen_losses[-show_step:]) / show_step
        crit_mean = sum(crit_losses[-show_step]) / show_step
        print(f"Epoch: {epoch}: Step {cur_step}: Generator loss: {gen_mean}, critic loss: {crit_mean}")

        plt.plot(
            range(len(gen_losses)),
            torch.Tensor(gen_losses),
            label("Generator Loss")
        ) 
        plt.plot(
            range(len(gen_losses)),
            torch.Tensor(crit_losses),
            label("Critic Loss")
        )
        plt.ylim(-1000,1000)
        plt.legend()
        plt.show()

    cur_step+=1

In [ ]:
#### Generate new faces
noise = gen_noise(batch_size, z_dim)
fake = gen(noise)
show(fake)

In [ ]:
# How to select a specific image
plt.imshow(fake[16].detach().cpu().permute(1,2,0).squeeze().clip(0,1))

In [ ]:
# MORPHING, interpolation between points in latent space
from mpl_toolkits.axes_grid1 import ImageGrid

gen_set=[]
z_shape=[1,200,1,1]
rows=4
steps=17

for i in range(rows):
    z1,z2 = torch.random(z_shape), torch.randn(z_shape)
    for alpha in np.linspace(0,1,steps):
        z=alpha*z1 + (1-alpha)*z2
        res=gen(z.cuda())[0]
        gen_set.append(res)

fig = plt.figure(figsize=(25,11))
grid = ImageGrid(fig, 111, nrows_ncols=(rows,steps), axes_pad=0.1)

for ax , img in zip (grid, gen_set):
    ax.axis('off')
    res=img.cpu().detach().permute(1,2,0)
    res=res-res.min()
    res=res/(res.max()-res.min())
    ax.imshow(res.clip(0,1.0))

plt.show()

In [ ]:
nn.ConvTranspose2d: (n-1)*stride -2*padding + ks
nn.Conv2d: (n=2*pad-ks)//stride +1